In [1]:
import pandas as pd
import numpy as np
import datetime
customers_df = pd.read_csv("assessment_customers.csv")
subscriptions_df = pd.read_csv("assessment_subscriptions.csv")
charges_df = pd.read_csv("assessment_charges.csv")
# Create a timestamp to keep help determine the length of the active subscriptions
timestamp = (datetime.datetime(2023, 2, 23))
time_stamp = timestamp.today()
today = time_stamp.strftime('%m/%d/%Y %X')
today

'02/28/2023 12:35:52'

In [2]:
# Replace the null values of the "last_active_month" column with "today" date values
# for only rows with null values for "cancelled_at", fill them with 0. Later we can select all CANCELLED status rows that do not lack a cancelled_at date.
# The CANCELLED status rows with missing "cancelled_at" dates will be voided in this assessment.
subscriptions_df.loc[subscriptions_df['status'] == 'CANCELLED', "cancelled_at"]= subscriptions_df.loc[subscriptions_df['status'] == 'CANCELLED', "cancelled_at"].fillna(0)
# Create a "last_active_month" column that contains an upper limit for the calculation of each active subscription's duration
subscriptions_df["last_active_month"] = subscriptions_df["cancelled_at"]
# Replace the null values of the "last_active_month" column with "today" date values. This will only replace the null values in the "ACTIVE" status rows.
subscriptions_df["last_active_month"] = subscriptions_df["last_active_month"].fillna(today)

In [3]:
# Create the parental dataframe that will contain all valid data, according to our assumptions.
# Only operate on subscriptions that are either active, or have been cancelled and have a valid "vancelled_at" date.
subscriptions_df=subscriptions_df.loc[(subscriptions_df["status"]=='ACTIVE')|(subscriptions_df["cancelled_at"]!=0)]

In [4]:
# convert these columns to datetime datatype
subscriptions_df['created_at'] = pd.to_datetime(subscriptions_df['created_at'])
subscriptions_df['last_active_month'] = pd.to_datetime(subscriptions_df['last_active_month'])



In [5]:
# Create column "active_months" which will contain a list of all the months in which a subscription was active
active_months = np.vectorize(lambda x, y: pd.period_range(x, y, freq='M').to_list())
subscriptions_df['active_months'] = active_months(subscriptions_df['created_at'], subscriptions_df['last_active_month'])

In [6]:
# This is the parental dataframe for problems 1 and 2.

subscriptions_df

,subscription_id,customer_id,address_id,status,product_title,recurring_price,quantity,created_at,cancelled_at,last_active_month,active_months
0,155517507,65434304,69830954,CANCELLED,Adrenal Super Tonic,26.78,1,2021-05-21 16:18:00,7/26/21 18:40,2021-07-26 18:40:00,"[2021-05, 2021-06, 2021-07]"
1,155517510,65434304,69830954,CANCELLED,Activated Charcoal,22.05,1,2021-05-21 16:18:00,7/26/21 18:40,2021-07-26 18:40:00,"[2021-05, 2021-06, 2021-07]"
2,155517518,65434304,69830954,CANCELLED,Golden Mind,26.25,1,2021-05-21 16:18:00,7/26/21 18:40,2021-07-26 18:40:00,"[2021-05, 2021-06, 2021-07]"
3,155517524,65434304,69830954,CANCELLED,Elderberry Defense,22.05,1,2021-05-21 16:18:00,7/26/21 18:40,2021-07-26 18:40:00,"[2021-05, 2021-06, 2021-07]"
4,155517525,65434304,69830954,CANCELLED,Longevity Mushrooms,25.20,1,2021-05-21 16:18:00,7/26/21 18:40,2021-07-26 18:40:00,"[2021-05, 2021-06, 2021-07]"
...,...,...,...,...,...,...,...,...,...,...,...
339020,335171272,106388353,116607381,ACTIVE,Adrenal Super Tonic,40.60,1,2023-02-20 14:50:00,NaN,2023-02-28 12:35:52,[2023-02]
339021,335171273,106388353,116607381,ACTIVE,Magnesium L-Threonate,47.60,1,2023-02-20 14:50:00,NaN,2023-02-28 12:35:52,[2023-02]
339022,335171274,106388353,116607381,ACTIVE,Liposomal Glutathione,61.60,1,2023-02-20 14:50:00,NaN,2023-02-28 12:35:52,[2023-02]
339023,335171275,106388353,116607381,ACTIVE,Mineral Shilajit,46.90,1,2023-02-20 14:50:00,NaN,2023-02-28 12:35:52,[2023-02]


In [7]:
# Use pandas explode() function to create a row for every month in which a subscriptions was active
active_subs = subscriptions_df.explode('active_months')

In [8]:
active_subs

,subscription_id,customer_id,address_id,status,product_title,recurring_price,quantity,created_at,cancelled_at,last_active_month,active_months
0,155517507,65434304,69830954,CANCELLED,Adrenal Super Tonic,26.78,1,2021-05-21 16:18:00,7/26/21 18:40,2021-07-26 18:40:00,2021-05
0,155517507,65434304,69830954,CANCELLED,Adrenal Super Tonic,26.78,1,2021-05-21 16:18:00,7/26/21 18:40,2021-07-26 18:40:00,2021-06
0,155517507,65434304,69830954,CANCELLED,Adrenal Super Tonic,26.78,1,2021-05-21 16:18:00,7/26/21 18:40,2021-07-26 18:40:00,2021-07
1,155517510,65434304,69830954,CANCELLED,Activated Charcoal,22.05,1,2021-05-21 16:18:00,7/26/21 18:40,2021-07-26 18:40:00,2021-05
1,155517510,65434304,69830954,CANCELLED,Activated Charcoal,22.05,1,2021-05-21 16:18:00,7/26/21 18:40,2021-07-26 18:40:00,2021-06
...,...,...,...,...,...,...,...,...,...,...,...
339020,335171272,106388353,116607381,ACTIVE,Adrenal Super Tonic,40.60,1,2023-02-20 14:50:00,NaN,2023-02-28 12:35:52,2023-02
339021,335171273,106388353,116607381,ACTIVE,Magnesium L-Threonate,47.60,1,2023-02-20 14:50:00,NaN,2023-02-28 12:35:52,2023-02
339022,335171274,106388353,116607381,ACTIVE,Liposomal Glutathione,61.60,1,2023-02-20 14:50:00,NaN,2023-02-28 12:35:52,2023-02
339023,335171275,106388353,116607381,ACTIVE,Mineral Shilajit,46.90,1,2023-02-20 14:50:00,NaN,2023-02-28 12:35:52,2023-02


In [9]:
# Use this dataframe to solve problem #2: The number of active subscriptions per customer per month
active_subs_df=active_subs.loc[subscriptions_df["status"]=='ACTIVE']



In [10]:
active_subs_df

,subscription_id,customer_id,address_id,status,product_title,recurring_price,quantity,created_at,cancelled_at,last_active_month,active_months
80,155962830,65537801,69942927,ACTIVE,ReGenesis Glutathione,56.32,1,2021-05-23 18:32:00,NaN,2023-02-28 12:35:52,2021-05
80,155962830,65537801,69942927,ACTIVE,ReGenesis Glutathione,56.32,1,2021-05-23 18:32:00,NaN,2023-02-28 12:35:52,2021-06
80,155962830,65537801,69942927,ACTIVE,ReGenesis Glutathione,56.32,1,2021-05-23 18:32:00,NaN,2023-02-28 12:35:52,2021-07
80,155962830,65537801,69942927,ACTIVE,ReGenesis Glutathione,56.32,1,2021-05-23 18:32:00,NaN,2023-02-28 12:35:52,2021-08
80,155962830,65537801,69942927,ACTIVE,ReGenesis Glutathione,56.32,1,2021-05-23 18:32:00,NaN,2023-02-28 12:35:52,2021-09
...,...,...,...,...,...,...,...,...,...,...,...
339020,335171272,106388353,116607381,ACTIVE,Adrenal Super Tonic,40.60,1,2023-02-20 14:50:00,NaN,2023-02-28 12:35:52,2023-02
339021,335171273,106388353,116607381,ACTIVE,Magnesium L-Threonate,47.60,1,2023-02-20 14:50:00,NaN,2023-02-28 12:35:52,2023-02
339022,335171274,106388353,116607381,ACTIVE,Liposomal Glutathione,61.60,1,2023-02-20 14:50:00,NaN,2023-02-28 12:35:52,2023-02
339023,335171275,106388353,116607381,ACTIVE,Mineral Shilajit,46.90,1,2023-02-20 14:50:00,NaN,2023-02-28 12:35:52,2023-02


In [11]:
# this is how you get the list view of this table, so you can see a total of the number of active subs per customer per month
active_subs_per_customer_per_month= active_subs_df.groupby(["active_months","customer_id"]).sum()[["quantity"]]

In [12]:
# rename the quantity column to quantity_active_subscriptions
active_subs_per_customer_per_month=active_subs_per_customer_per_month.rename(columns={"quantity":"quantity_active_subscriptions"})

In [13]:
############# SOLUTION TO Question #2: Number of Active Subscriptions per Customer Per Month###########################################################################################
active_subs_per_customer_per_month

quantity_active_subscriptions
active_months customer_id                               
2021-05       65537801                                 3
              65570608                                 4
              65571284                                 1
              65572010                                 5
              65573050                                 3
...                                                  ...
2023-02       106387265                                2
              106387693                                5
              106387948                                1
              106388291                                1
              106388353                                7

[116785 rows x 1 columns]

In [14]:
# Export the pivot table to an excel csv file.
active_subs_per_customer_per_month.to_csv("active_subs_per_customer_per_month.csv")

In [15]:
# Total number of active customers per month
active_subs_df.groupby("active_months").nunique()["customer_id"]

active_months
2021-05       19
2021-06      363
2021-07      398
2021-08      469
2021-09      565
2021-10      702
2021-11      884
2021-12     1009
2022-01     1329
2022-02     1611
2022-03     2097
2022-04     2992
2022-05     3742
2022-06     4534
2022-07     5326
2022-08     6338
2022-09     7675
2022-10     9210
2022-11    12047
2022-12    14474
2023-01    18639
2023-02    22362
Freq: M, Name: customer_id, dtype: int64

In [16]:
# Now begin to solve the percentage of active customers that order

charges_df

,charge_id,address_id,customer_id,external_order_id,type,status,created_at,processed_at,subtotal_price,total_taxes,total_shipping,total_discounts,total_price,total_refunds
0,645673217,102609721.0,93942725.0,4.707530e+12,CHECKOUT,SUCCESS,2022-09-01 0:02:47,2022-09-01 0:02:48,76.16,0.00,9.54,13.44,85.70,NaN
1,645673217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,645752162,102611159.0,66791434.0,4.707625e+12,CHECKOUT,SUCCESS,2022-09-01 0:46:37,2022-09-01 0:46:38,818.14,59.31,12.51,144.36,889.96,NaN
3,645752162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,645752162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225395,723715170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
225396,723716752,112313752.0,102348755.0,4.891283e+12,CHECKOUT,SUCCESS,2022-12-30 23:47:22,2022-12-30 23:47:22,45.23,0.00,0.00,15.07,45.23,NaN
225397,723718515,112314170.0,102349107.0,4.891288e+12,CHECKOUT,SUCCESS,2022-12-30 23:58:23,2022-12-30 23:58:23,80.33,0.00,0.00,26.77,80.33,NaN
225398,723718515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
ordering_customers = charges_df.loc[(charges_df["type"]=='CHECKOUT')&(charges_df["status"]=='SUCCESS')]
ordering_customers["customer_id"]=ordering_customers["customer_id"].astype('int')
ordering_customers["customer_id"]=ordering_customers["customer_id"].astype('str')
ordering_customers

/Applications/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Applications/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,charge_id,address_id,customer_id,external_order_id,type,status,created_at,processed_at,subtotal_price,total_taxes,total_shipping,total_discounts,total_price,total_refunds
0,645673217,102609721.0,93942725,4.707530e+12,CHECKOUT,SUCCESS,2022-09-01 0:02:47,2022-09-01 0:02:48,76.16,0.00,9.54,13.44,85.70,NaN
2,645752162,102611159.0,66791434,4.707625e+12,CHECKOUT,SUCCESS,2022-09-01 0:46:37,2022-09-01 0:46:38,818.14,59.31,12.51,144.36,889.96,NaN
21,645758945,102611269.0,81569039,4.707632e+12,CHECKOUT,SUCCESS,2022-09-01 0:50:28,2022-09-01 0:50:29,152.60,0.00,66.25,152.60,218.85,NaN
27,645780331,102611615.0,93945455,4.707654e+12,CHECKOUT,SUCCESS,2022-09-01 1:02:12,2022-09-01 1:02:13,129.42,9.39,0.00,22.83,138.81,NaN
32,645802196,102611796.0,93945610,4.707674e+12,CHECKOUT,SUCCESS,2022-09-01 1:09:56,2022-09-01 1:09:56,194.57,16.06,0.00,34.33,210.63,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225383,723710374,112312331.0,102347496,4.891268e+12,CHECKOUT,SUCCESS,2022-12-30 23:10:56,2022-12-30 23:10:56,75.22,0.00,0.00,25.08,75.22,NaN
225386,723712540,112312811.0,102347918,4.891273e+12,CHECKOUT,SUCCESS,2022-12-30 23:22:41,2022-12-30 23:22:42,93.60,6.08,0.00,31.20,99.68,NaN
225390,723715170,112313404.0,94661481,4.891280e+12,CHECKOUT,SUCCESS,2022-12-30 23:37:36,2022-12-30 23:37:36,157.29,0.00,0.00,67.41,157.29,NaN
225396,723716752,112313752.0,102348755,4.891283e+12,CHECKOUT,SUCCESS,2022-12-30 23:47:22,2022-12-30 23:47:22,45.23,0.00,0.00,15.07,45.23,NaN


In [18]:
ordering_customers['created_at'] = pd.to_datetime(ordering_customers['created_at']).dt.to_period('M')
ordering_customers["created_at"]=ordering_customers["created_at"].astype('str')
ordering_customers

/Applications/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Applications/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,charge_id,address_id,customer_id,external_order_id,type,status,created_at,processed_at,subtotal_price,total_taxes,total_shipping,total_discounts,total_price,total_refunds
0,645673217,102609721.0,93942725,4.707530e+12,CHECKOUT,SUCCESS,2022-09,2022-09-01 0:02:48,76.16,0.00,9.54,13.44,85.70,NaN
2,645752162,102611159.0,66791434,4.707625e+12,CHECKOUT,SUCCESS,2022-09,2022-09-01 0:46:38,818.14,59.31,12.51,144.36,889.96,NaN
21,645758945,102611269.0,81569039,4.707632e+12,CHECKOUT,SUCCESS,2022-09,2022-09-01 0:50:29,152.60,0.00,66.25,152.60,218.85,NaN
27,645780331,102611615.0,93945455,4.707654e+12,CHECKOUT,SUCCESS,2022-09,2022-09-01 1:02:13,129.42,9.39,0.00,22.83,138.81,NaN
32,645802196,102611796.0,93945610,4.707674e+12,CHECKOUT,SUCCESS,2022-09,2022-09-01 1:09:56,194.57,16.06,0.00,34.33,210.63,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225383,723710374,112312331.0,102347496,4.891268e+12,CHECKOUT,SUCCESS,2022-12,2022-12-30 23:10:56,75.22,0.00,0.00,25.08,75.22,NaN
225386,723712540,112312811.0,102347918,4.891273e+12,CHECKOUT,SUCCESS,2022-12,2022-12-30 23:22:42,93.60,6.08,0.00,31.20,99.68,NaN
225390,723715170,112313404.0,94661481,4.891280e+12,CHECKOUT,SUCCESS,2022-12,2022-12-30 23:37:36,157.29,0.00,0.00,67.41,157.29,NaN
225396,723716752,112313752.0,102348755,4.891283e+12,CHECKOUT,SUCCESS,2022-12,2022-12-30 23:47:22,45.23,0.00,0.00,15.07,45.23,NaN


In [19]:
ordering_customers["customer_order"]=ordering_customers["created_at"]+ordering_customers["customer_id"]
ordering_customers

/Applications/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,charge_id,address_id,customer_id,external_order_id,type,status,created_at,processed_at,subtotal_price,total_taxes,total_shipping,total_discounts,total_price,total_refunds,customer_order
0,645673217,102609721.0,93942725,4.707530e+12,CHECKOUT,SUCCESS,2022-09,2022-09-01 0:02:48,76.16,0.00,9.54,13.44,85.70,NaN,2022-0993942725
2,645752162,102611159.0,66791434,4.707625e+12,CHECKOUT,SUCCESS,2022-09,2022-09-01 0:46:38,818.14,59.31,12.51,144.36,889.96,NaN,2022-0966791434
21,645758945,102611269.0,81569039,4.707632e+12,CHECKOUT,SUCCESS,2022-09,2022-09-01 0:50:29,152.60,0.00,66.25,152.60,218.85,NaN,2022-0981569039
27,645780331,102611615.0,93945455,4.707654e+12,CHECKOUT,SUCCESS,2022-09,2022-09-01 1:02:13,129.42,9.39,0.00,22.83,138.81,NaN,2022-0993945455
32,645802196,102611796.0,93945610,4.707674e+12,CHECKOUT,SUCCESS,2022-09,2022-09-01 1:09:56,194.57,16.06,0.00,34.33,210.63,NaN,2022-0993945610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225383,723710374,112312331.0,102347496,4.891268e+12,CHECKOUT,SUCCESS,2022-12,2022-12-30 23:10:56,75.22,0.00,0.00,25.08,75.22,NaN,2022-12102347496
225386,723712540,112312811.0,102347918,4.891273e+12,CHECKOUT,SUCCESS,2022-12,2022-12-30 23:22:42,93.60,6.08,0.00,31.20,99.68,NaN,2022-12102347918
225390,723715170,112313404.0,94661481,4.891280e+12,CHECKOUT,SUCCESS,2022-12,2022-12-30 23:37:36,157.29,0.00,0.00,67.41,157.29,NaN,2022-1294661481
225396,723716752,112313752.0,102348755,4.891283e+12,CHECKOUT,SUCCESS,2022-12,2022-12-30 23:47:22,45.23,0.00,0.00,15.07,45.23,NaN,2022-12102348755


In [20]:
customers_who_order=pd.DataFrame()
# isolate the customer_order Id's from the charges_df dataframe
customers_who_order["customer_order"] = ordering_customers["customer_order"].unique()
customers_who_order.sort_values("customer_order")
customers_who_order # this is from the ordering_customers dataframe

,customer_order
0,2022-0993942725
1,2022-0966791434
2,2022-0981569039
3,2022-0993945455
4,2022-0993945610
...,...
18980,2022-12102347496
18981,2022-12102347918
18982,2022-1294661481
18983,2022-12102348755


In [21]:
active_subs_df["active_months"]=active_subs_df["active_months"].astype('str')
active_subs_df["customer_id"]=active_subs_df["customer_id"].astype('str')

/Applications/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Applications/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
active_subs_df["customer_order"]=active_subs_df["active_months"]+active_subs_df["customer_id"]
active_subs_df


/Applications/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,subscription_id,customer_id,address_id,status,product_title,recurring_price,quantity,created_at,cancelled_at,last_active_month,active_months,customer_order
80,155962830,65537801,69942927,ACTIVE,ReGenesis Glutathione,56.32,1,2021-05-23 18:32:00,NaN,2023-02-28 12:35:52,2021-05,2021-0565537801
80,155962830,65537801,69942927,ACTIVE,ReGenesis Glutathione,56.32,1,2021-05-23 18:32:00,NaN,2023-02-28 12:35:52,2021-06,2021-0665537801
80,155962830,65537801,69942927,ACTIVE,ReGenesis Glutathione,56.32,1,2021-05-23 18:32:00,NaN,2023-02-28 12:35:52,2021-07,2021-0765537801
80,155962830,65537801,69942927,ACTIVE,ReGenesis Glutathione,56.32,1,2021-05-23 18:32:00,NaN,2023-02-28 12:35:52,2021-08,2021-0865537801
80,155962830,65537801,69942927,ACTIVE,ReGenesis Glutathione,56.32,1,2021-05-23 18:32:00,NaN,2023-02-28 12:35:52,2021-09,2021-0965537801
...,...,...,...,...,...,...,...,...,...,...,...,...
339020,335171272,106388353,116607381,ACTIVE,Adrenal Super Tonic,40.60,1,2023-02-20 14:50:00,NaN,2023-02-28 12:35:52,2023-02,2023-02106388353
339021,335171273,106388353,116607381,ACTIVE,Magnesium L-Threonate,47.60,1,2023-02-20 14:50:00,NaN,2023-02-28 12:35:52,2023-02,2023-02106388353
339022,335171274,106388353,116607381,ACTIVE,Liposomal Glutathione,61.60,1,2023-02-20 14:50:00,NaN,2023-02-28 12:35:52,2023-02,2023-02106388353
339023,335171275,106388353,116607381,ACTIVE,Mineral Shilajit,46.90,1,2023-02-20 14:50:00,NaN,2023-02-28 12:35:52,2023-02,2023-02106388353


In [30]:
customer_order_list = active_subs_df.groupby(["active_months","customer_order"]).sum()[["quantity"]]
unique_active_customers_list=customer_order_list.reset_index()[["active_months","customer_order"]]
unique_active_customers_list

,active_months,customer_order
0,2021-05,2021-0565537801
1,2021-05,2021-0565570608
2,2021-05,2021-0565571284
3,2021-05,2021-0565572010
4,2021-05,2021-0565573050
...,...,...
116780,2023-02,2023-0299995282
116781,2023-02,2023-0299995746
116782,2023-02,2023-0299995989
116783,2023-02,2023-0299996403


In [32]:
percent_active_customers_who_order_calc = pd.merge(unique_active_customers_list,customers_who_order, on="customer_order")
percent_active_customers_who_order_calc

,active_months,customer_order
0,2022-09,2022-0965468909
1,2022-09,2022-0965582042
2,2022-09,2022-0965584370
3,2022-09,2022-0965633378
4,2022-09,2022-0965637387
...,...,...
8873,2022-12,2022-1299932991
8874,2022-12,2022-1299950316
8875,2022-12,2022-1299960853
8876,2022-12,2022-1299977468


In [38]:
active_ordering_customer_per_month = percent_active_customers_who_order_calc.groupby("active_months").nunique()[["customer_order"]]
active_ordering_customer_per_month

,customer_order
active_months,
2022-09,1479
2022-10,1624
2022-11,3220
2022-12,2555


In [39]:
total_active_customers = active_subs_df.groupby("active_months").nunique()[["customer_id"]]
total_active_customers

,customer_id
active_months,
2021-05,19
2021-06,363
2021-07,398
2021-08,469
2021-09,565
2021-10,702
2021-11,884
2021-12,1009
2022-01,1329


In [40]:
percent_of_active_customers_who_order_each_month=100*active_ordering_customer_per_month["customer_order"]/total_active_customers["customer_id"]


In [41]:
############# SOLUTION TO Question #3 Percengtage of Active Customers Who Order Each Month ###########################################################################################
percent_of_active_customers_who_order_each_month=pd.DataFrame(percent_of_active_customers_who_order_each_month)
percent_of_active_customers_who_order_each_month = percent_of_active_customers_who_order_each_month.rename(columns={0:"%_active_customers_who_order_each_month"})
percent_of_active_customers_who_order_each_month
percent_of_active_customers_who_order_each_month.reset_index()
percent_of_active_customers_who_order_each_month=percent_of_active_customers_who_order_each_month[16:20]

In [42]:
percent_of_active_customers_who_order_each_month

,%_active_customers_who_order_each_month
active_months,
2022-09,19.270358
2022-10,17.633008
2022-11,26.728646
2022-12,17.652342
